In [13]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [14]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNDU5Yjg5ZTItNTQyYS00ZGJiLWEwMmUtMTMwYTJjZThhYTY3IiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJyZWNpcGllbnRLZXlzIjogWyJCYW85RVhXVkdBc0Rvd25EYzJaRTg5NlJxOVRvZ2RFYVk1TVhma2M5YkRpQiJdLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly9hZ2VudDEubG9jYWxob3N0In0=


In [15]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMjliNDUwMmQtMzVlMS00MGQ4LTg5ZTEtMDc0NTM3NDNkYmY1IiwgInJlY2lwaWVudEtleXMiOiBbIkVQWllhR1ZheXVMc2lUdjk5QWpBcjd2VGl2eXNZNWFTQW1UbXpiczhOelhLIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50Mi5sb2NhbGhvc3QiLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ==


In [16]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '29b4502d-35e1-40d8-89e1-07453743dbf5',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['EPZYaGVayuLsiTv99AjAr7vTivysY5aSAmTmzbs8NzXK'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [17]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 3TZJer3f57CBpodHhLjXwpZS4cHJy7pKb99ury9SagZfCtXY4K4UpAHHUozvXhMnYESgU8k51g9xEuhaJYXBkXG3

Public key: 
 BZyLGEbXZcAUrBGhRJekrASZFdLzZMpZ2VxGLcMs7Mnf


**Sending a connection request to acapy**

In [18]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [19]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [20]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

In [21]:
connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

34e573c4-1ff8-4921-93bb-d001c917baa6 
 e9d5eaf8-ee4a-483f-8c3a-c8aef7811a75


### FEATURE DISCOVERY

In [22]:
# pp.pprint(util.getFeatureDiscovery(agents[0]))

In [23]:
# consentSchema: function() {
#       return {
#         namespace: "consent",
#         DRI: "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s"
#       }
#     },

# consentAnswers: function() {
#       return [
#         "zQmNeoZGfo1chKACu9RLrVoG4afjLrGnAz1rhsfvGZwg9Z3",
#         "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"
#       ]

In [24]:
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "data_dri": "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"
  }
})
print(add.text)
add = json.loads(add.text)

{"service_schema": {"oca_schema_namespace": "string", "oca_schema_dri": "string"}, "consent_schema": {"oca_schema_namespace": "consent", "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", "data_dri": "zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"}, "label": "consent_test", "created_at": "2020-09-03 10:35:09.346569Z", "updated_at": "2020-09-03 10:35:09.346569Z"}


In [146]:
# did = requests.post('http://localhost:8151/wallet/did/create')
# did = json.loads(did.text)
# print(did)

In [147]:
# create_did = requests.post('http://localhost:8151/wallet/did/create')
# create_did = json.loads(create_did.text)
# print(create_did)

In [125]:
# register_did = requests.post('http://localhost:8151/ledger/register-nym', params={
#     "verkey": create_did['result']['verkey'],
#     "did": create_did['result']['did'],
# })
# register_did = json.loads(register_did.text)
# print(register_did)

In [126]:
# did = requests.post('http://localhost:8151/wallet/did/public', params={
#     'did': register_did['result']['did']
# })
# did = json.loads(did.text)
# print(did)

In [127]:
## DID

In [128]:
newDid1 = util.createDid(agents[1])
util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])

KeyboardInterrupt: 

In [129]:
newDid0 = util.createDid(agents[0])
util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])

KeyboardInterrupt: 

In [130]:
util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': '5d6097f3-216e-4440-b268-ed0b6990150f',
 '~thread': {'thid': '9977f22a-4471-4431-ba55-9375f24c9b4c'},
 'result': {'metadata': {'public': True, 'posted': True},
  'did': 'UPgDE2mVJDejCYne79etpP',
  'verkey': 'FvtBYTQH7gKK7Mnda4g1r6rCiuy1L3YeHwGfpHEekpcB'}}

In [25]:
service_list = requests.get('http://localhost:8150/verifiable-services/request-service-list/' + connection[0])
print(service_list)
service_list = json.loads(service_list.text)
pp.pprint(service_list)

<Response [200]>
'SUCCESS: request sent, expect a webhook notification'


In [26]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
service_list = json.loads(service_list.text)
pp.pprint(service_list)

{'connection_id': '34e573c4-1ff8-4921-93bb-d001c917baa6',
 'created_at': '2020-09-03 10:35:26.098661Z',
 'services': [{'consent_schema': {'data_dri': 'zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': 'c7ae1a9c-dcaa-4fba-914c-ff2e6b29a38d',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}},
              {'consent_schema': {'data_dri': 'zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': 'd748246d-0a74-4c7c-8dee-24748b5009e0',
            

In [133]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
  "payload": "abc",
  "service": service_list['services'][0],
})
apply = json.loads(apply.text)
print(apply)

JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [87]:
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue-self', json={
    "exchange_id": apply['exchange_id'],
})
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

[{'author': 'other',
  'connection_id': 'f9da8cf9-e6b7-4611-a617-803b4d522e50',
  'consent_schema': '{"oca_schema_namespace": "consent", "oca_schema_dri": '
                    '"fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", '
                    '"data_dri": '
                    '"zQmPsU57nqWY8jzndU9AE2RK4EXvjMLGmytVpUNxfRpm18G"}',
  'created_at': '2020-08-27 08:24:22.427200Z',
  'exchange_id': '6ff39f28-fe14-4458-8a80-bd320e75c0ce',
  'issue_id': '9dbf4c103a3fe09a9ac12f93e5798cfc6759cf1518a7688318dbc79750f2cc1a',
  'label': 'consent_test',
  'payload': None,
  'service_id': '93230419-4309-4c2b-a354-43b7d221791b',
  'service_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string"}',
  'state': 'pending',
  'updated_at': '2020-08-27 08:24:22.427200Z'}]


In [88]:
issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
{'credential_exchange_record': {'auto_issue': True,
                                'auto_offer': False,
                                'auto_remove': False,
                                'connection_id': 'f9da8cf9-e6b7-4611-a617-803b4d522e50',
                                'created_at': '2020-08-27 08:24:34.592887Z',
                                'credential_definition_id': 'BEy6G3VuWpAhttFUj966Ud:3:CL:637:Services',
                                'credential_exchange_id': '3413279f-de9e-4856-8690-641f25c33bad',
                                'credential_offer': {'cred_def_id': 'BEy6G3VuWpAhttFUj966Ud:3:CL:637:Services',
                                                     'key_correctness_proof': {'c': '13707960370824359913102657843276314434517489867794612769013494427286637412795',
                                                                               'xr_cap': [['oca_schema_namespace',
                                                                

In [36]:
payload = "some string yes dd"
payload_response = "response yay"
hash_id = hashlib.sha256(payload.encode("UTF-8")).hexdigest()
hash_id_response = hashlib.sha256(payload_response.encode("UTF-8")).hexdigest()
print(payload, hash_id)

some string yes dd b7c34035833ad394aaf721e431ae4a7f6359d66e1c69e8be9ec4c3e1d2e1ea1f


In [293]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': [{'attrs': {'consent_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string", '
                                          '"data_dri": "string"}',
                        'label': 'memxsassdsssdfdeds',
                        'service_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string"}'},
              'cred_def_id': 'B6hy9V4FLjh6tQmknfEvTt:3:CL:273:Services',
              'cred_rev_id': None,
              'referent': 'e56cb0b5-f044-4176-b4d8-246c27ea0d00',
              'rev_reg_id': None,
              'schema_id': 'B6hy9V4FLjh6tQmknfEvTt:2:memxsassdsssdfdeds:1.0'},
             {'attrs': {'consent_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string", '
                                          '"data_dri": "string"}',
                        'label': 'memxsassdsssdfdeds',
         

I added returning hash id to add schema, I need to test if it works with both rejecting (not found schema) and accepting schema dound

### Add schema to storage

In [52]:
add = requests.post('http://localhost:8150/schema-storage/add', json={
  "payload": "test"
})
print(add.text)
add = json.loads(add.text)

{"hash_id": "9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08", "payload": "test"}


### SEND Initial Request

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)